# 世界の最新の感染者数

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
del df["Lat"]
del df["Long"]

In [3]:
df.head()

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,6/16/21,6/17/21,6/18/21,6/19/21,6/20/21,6/21/21,6/22/21,6/23/21,6/24/21,6/25/21
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,93288,96531,98734,98734,98734,103902,105749,107957,109532,111592
1,NaN,Albania,0,0,0,0,0,0,0,0,...,132476,132481,132484,132488,132490,132490,132496,132497,132499,132506
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,134458,134840,135219,135586,135821,136294,136679,137049,137403,137772
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,13836,13839,13842,13842,13842,13864,13864,13873,13877,13882
4,NaN,Angola,0,0,0,0,0,0,0,0,...,37094,37289,37467,37604,37678,37748,37874,38002,38091,38371


In [4]:
latest_date = df.tail(1).columns[-1]
latest_date

'6/25/21'

In [5]:
latest_column_key = df.columns[-1]
latest_column_key2 = df.columns[-2]
df = df[['Province/State','Country/Region',latest_column_key2,latest_column_key]]
df = df.rename(columns={latest_column_key:"latest_confirmed"})
df = df.rename(columns={latest_column_key2:"latest_confirmed2"})
df["new_confirmed"] = df.latest_confirmed - df.latest_confirmed2
df.head()

,Province/State,Country/Region,latest_confirmed2,latest_confirmed,new_confirmed
0,NaN,Afghanistan,109532,111592,2060
1,NaN,Albania,132499,132506,7
2,NaN,Algeria,137403,137772,369
3,NaN,Andorra,13877,13882,5
4,NaN,Angola,38091,38371,280


## 全世界の最新の感染者数

In [6]:
print("全世界感染者数({}時点):{:,}人".format(latest_date,df["latest_confirmed"].sum()))
print("全世界感染者数平均({}時点):{:,}人".format(latest_date,df["latest_confirmed"].mean()))
print("全世界の新規感染者数({}時点):{:,}人".format(latest_date,df["new_confirmed"].sum()))
print("全世界の新規感染者数平均({}時点):{:,}人".format(latest_date,df["new_confirmed"].mean()))


全世界感染者数(6/25/21時点):180,410,080人
全世界感染者数平均(6/25/21時点):646,631.1111111111人
全世界の新規感染者数(6/25/21時点):410,692人
全世界の新規感染者数平均(6/25/21時点):1,472.0143369175628人


## 国別の最新の感染者数

In [7]:
country = df.groupby(["Country/Region"],as_index=False).sum("latest_confirmed")
country = country.sort_values("latest_confirmed", ascending=False)

In [8]:
country.head()

,Country/Region,latest_confirmed2,latest_confirmed,new_confirmed
181,US,33590481,33603148,12667
79,India,30134445,30183143,48698
23,Brazil,18243483,18322760,79277
62,France,5826278,5828266,1988
180,Turkey,5393248,5398878,5630


In [9]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [10]:
plt.figure(figsize=(9.5,5))
plt.title("Worst 50")
plt.xticks(rotation="90",fontsize=9)
plt.ylabel("count (1000 people)")
#fig.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
plt.bar(country.head(50)["Country/Region"],country.head(50)["latest_confirmed"]/1000,label="positive")
plt.legend()
plt.tight_layout()

<IPython.core.display.Javascript object>

## ヒストグラム 

In [11]:
plt.figure(figsize=(10,5))
plt.title("histgram(million)")
#plt.xticks(rotation="90",fontsize=9)
#plt.ylabel("count (1000 people)")
#fig.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
plt.hist(country["latest_confirmed"]/1000000,bins=20)

<IPython.core.display.Javascript object>

(array([175.,   8.,   6.,   3.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,
          0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   1.]),
 array([ 0.       ,  1.6801574,  3.3603148,  5.0404722,  6.7206296,
         8.400787 , 10.0809444, 11.7611018, 13.4412592, 15.1214166,
        16.801574 , 18.4817314, 20.1618888, 21.8420462, 23.5222036,
        25.202361 , 26.8825184, 28.5626758, 30.2428332, 31.9229906,
        33.603148 ]),
 <a list of 20 Patch objects>)

## 人口当たりの分析

In [12]:
pop = pd.read_excel("world-data/WPP2019_POP_F01_1_TOTAL_POPULATION_BOTH_SEXES.xlsx",header=16)
pop.head(1)

,Index,Variant,"Region, subregion, country or area *",Notes,Country code,Type,Parent code,1950,1951,1952,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,1,Estimates,WORLD,NaN,900,World,0,2536431.018,2584034.227,2630861.69,...,7041194.168,7125827.957,7210582.041,7295290.759,7379796.967,7464021.934,7547858.9,7631091.113,7713468.205,7794798.729


In [13]:
pop.iloc[:,-1]

0      7794798.729
1              ...
2      1273304.261
3      6521494.468
4      1057438.163
          ...     
284         62.273
285      37742.157
286         56.772
287          5.795
288     331002.647
Name: 2020, Length: 289, dtype: object

In [14]:
pop_latest = pop.loc[pop["Type"]=="Country/Area"][[pop.columns[2],pop.columns[-1]]]

In [15]:
pop_latest.columns = ["country","population"]

In [16]:
pop_latest.head()

,country,population
26,Burundi,11890.781
27,Comoros,869.595
28,Djibouti,988.002
29,Eritrea,3546.427
30,Ethiopia,114963.583


In [17]:
country.head()

,Country/Region,latest_confirmed2,latest_confirmed,new_confirmed
181,US,33590481,33603148,12667
79,India,30134445,30183143,48698
23,Brazil,18243483,18322760,79277
62,France,5826278,5828266,1988
180,Turkey,5393248,5398878,5630


### 世界全体での人口10万人あたり陽性者数

In [18]:
country.latest_confirmed.sum()

180410080

In [19]:
pop_latest.population.sum()

7794798.729

In [20]:
print("****世界全体での人口10万人あたり陽性者数:{}****".format(country.latest_confirmed.sum() / (pop_latest.population.sum() / 100)))

****世界全体での人口10万人あたり陽性者数:2314.4931161441923****


### 各国の人口１０万人あたり陽性者数

In [21]:
c_merged = pd.merge(country, pop_latest, left_on="Country/Region", right_on="country", how="left")
c_merged.loc[c_merged["country"].isna()]

,Country/Region,latest_confirmed2,latest_confirmed,new_confirmed,country,population
0,US,33590481,33603148,12667,NaN,NaN
5,Russia,5325940,5346005,20065,NaN,NaN
12,Iran,3140129,3150949,10820,NaN,NaN
46,Bolivia,429178,431360,2182,NaN,NaN
61,West Bank and Gaza,313015,313520,505,NaN,NaN
68,Venezuela,265642,266952,1310,NaN,NaN
73,Moldova,256387,256475,88,NaN,NaN
84,"Korea, South",153789,154457,668,NaN,NaN
85,Burma,150714,151573,859,NaN,NaN
95,Kosovo,107299,107299,0,NaN,NaN


In [22]:
p_merged = pd.merge(pop_latest, country, left_on="country", right_on="Country/Region", how="left")
len(p_merged.loc[p_merged["Country/Region"].isna()])

63

In [23]:
pop_latest.loc[pop_latest["country"].str.contains("Syria")]

,country,population
104,Syrian Arab Republic,17500.657


In [24]:
renames = {
    "US":"United States of America",
    "Russia":"Russian Federation",
    "Iran":"Iran (Islamic Republic of)",
    "Bolivia":"Bolivia (Plurinational State of)",
    "Moldova":"Republic of Moldova",
    "Venezuela":"Venezuela (Bolivarian Republic of)",
    "Korea, South":"Republic of Korea",
    "Congo (Kinshasa)":"Democratic Republic of the Congo",
    "Congo (Brazzaville)":"Congo",
    "Vietnam":"Viet Nam",
    "Laos":"Lao People's Democratic Republic",
    "Taiwan*":"China, Taiwan Province of China",
    "Tanzania":"United Republic of Tanzania",
    "Brunei":"Brunei Darussalam",
    "Micronesia":"Micronesia (Fed. States of)",
    "Syria":"Syrian Arab Republic",
}

In [25]:
country_renamed = country.copy()
country_renamed["Country/Region"].replace(renames,inplace=True)

In [26]:
c_merged = pd.merge(country_renamed, pop_latest, left_on="Country/Region", right_on="country", how="left")
c_merged.loc[c_merged["country"].isna()]

,Country/Region,latest_confirmed2,latest_confirmed,new_confirmed,country,population
61,West Bank and Gaza,313015,313520,505,NaN,NaN
85,Burma,150714,151573,859,NaN,NaN
95,Kosovo,107299,107299,0,NaN,NaN
112,Cote d'Ivoire,48058,48058,0,NaN,NaN
179,Diamond Princess,712,712,0,NaN,NaN
187,MS Zaandam,9,9,0,NaN,NaN
192,Summer Olympics 2020,1,1,0,NaN,NaN


In [27]:
c_merged["positives per 1000 population"] = c_merged["latest_confirmed"] / (c_merged["population"])
c_merged["positives per 100,000 population"] = (c_merged["latest_confirmed"] / (c_merged["population"]/100)).astype(float)
c_merged["new positives per 100,000 population"] = (c_merged["new_confirmed"] / (c_merged["population"]/100)).astype(float)

In [28]:
c_merged = c_merged.sort_values("positives per 1000 population", ascending=False)

In [29]:
c_merged.loc[c_merged["Country/Region"]=="United States of America"]

,Country/Region,latest_confirmed2,latest_confirmed,new_confirmed,country,population,positives per 1000 population,"positives per 100,000 population","new positives per 100,000 population"
0,United States of America,33590481,33603148,12667,United States of America,331002.647,101.519273,10151.927274,3.826858


In [30]:
fig = plt.figure(figsize=(12,8))

plt.subplot(1,2,1) # (rows, columns, panel number)
plt.title("positives per pupulation WORST 50")
#plt.xticks(rotation="90",fontsize=9)
plt.yticks(fontsize=9)
plt.xlabel("count per 100,000 population")
plt.grid(axis="x")
#fig.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
#plt.bar(c_merged.head(50)["Country/Region"],c_merged.head(50)["positives per 100,000 population"],label="positive")
chart_data=c_merged.head(50).sort_values("positives per 100,000 population",ascending=True)
plt.barh(chart_data["Country/Region"],chart_data["positives per 100,000 population"],label="positive")
#plt.text(0,chart_data["positives per 100,000 population"],chart_data["population"].astype(str))
#plt.text(0,chart_data["positives per 100,000 population"].all(),chart_data["population"].astype(str).all())
plt.legend()

plt.subplot(1,2,2) # (rows, columns, panel number)
plt.title("positives per pupulation BEST 50")
#plt.xticks(rotation="90",fontsize=9)
plt.yticks(fontsize=9)
plt.xlabel("count per 100,000 population")
plt.grid(axis="x")
#fig.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
c_asc = c_merged.sort_values("positives per 1000 population")
chart_data=c_asc.head(50).sort_values("positives per 100,000 population",ascending=False)
plt.barh(chart_data["Country/Region"],chart_data["positives per 100,000 population"],label="positive")
plt.legend()

fig.tight_layout()


<IPython.core.display.Javascript object>

In [31]:
fig.savefig("out/covid-global-tops.png")

In [32]:
plt.figure(figsize=(9.5,5))
plt.title("histgram(positives per 100,000 pop)")
plt.hist(c_merged["positives per 100,000 population"],bins=[0,2000,4000,8000,10000,12000,14000,16000,18000])

<IPython.core.display.Javascript object>

(array([97., 27., 33., 18.,  5.,  2.,  5.,  1.]),
 array([    0,  2000,  4000,  8000, 10000, 12000, 14000, 16000, 18000]),
 <a list of 8 Patch objects>)

In [33]:
c_merged.describe()

,latest_confirmed2,latest_confirmed,new_confirmed,"positives per 100,000 population","new positives per 100,000 population"
count,1.950000e+02,1.950000e+02,195.000000,188.000000,188.000000
mean,9.230738e+05,9.251799e+05,2106.112821,3560.333760,9.887887
std,3.568095e+06,3.573359e+06,7698.954005,4077.028850,38.745310
min,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
25%,1.240550e+04,1.247600e+04,5.000000,239.244799,0.141953
50%,1.036510e+05,1.036800e+05,88.000000,1872.284416,1.480348
75%,4.166830e+05,4.178075e+05,893.000000,6368.767898,7.846646
max,3.359048e+07,3.360315e+07,79277.000000,17966.737850,504.372585


In [34]:
c_merged.head()

,Country/Region,latest_confirmed2,latest_confirmed,new_confirmed,country,population,positives per 1000 population,"positives per 100,000 population","new positives per 100,000 population"
140,Andorra,13877,13882,5,Andorra,77.265,179.667379,17966.737850,6.471235
98,Montenegro,100131,100145,14,Montenegro,628.062,159.450819,15945.081855,2.229079
20,Czechia,1666521,1666686,165,Czechia,10708.982,155.634401,15563.440110,1.540763
69,Bahrain,264405,264693,288,Bahrain,1701.583,155.556914,15555.691377,16.925416
135,Seychelles,14620,15116,496,Seychelles,98.34,153.711613,15371.161277,504.372585


In [35]:
c_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195 entries, 140 to 192
Data columns (total 9 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Country/Region                        195 non-null    object 
 1   latest_confirmed2                     195 non-null    int64  
 2   latest_confirmed                      195 non-null    int64  
 3   new_confirmed                         195 non-null    int64  
 4   country                               188 non-null    object 
 5   population                            188 non-null    object 
 6   positives per 1000 population         188 non-null    object 
 7   positives per 100,000 population      188 non-null    float64
 8   new positives per 100,000 population  188 non-null    float64
dtypes: float64(2), int64(3), object(4)
memory usage: 15.2+ KB


In [36]:
plt.figure(figsize=(9.5,5))
plt.title("histgram(positives per 100,000 pop)")
plt.hist(c_merged["positives per 100,000 population"],bins=200)

<IPython.core.display.Javascript object>

/home/masaru/.pyenv/versions/anaconda3-2020.02/lib/python3.7/site-packages/numpy/lib/histograms.py:839: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
/home/masaru/.pyenv/versions/anaconda3-2020.02/lib/python3.7/site-packages/numpy/lib/histograms.py:840: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)


(array([26., 15., 11., 13.,  2.,  3.,  2.,  1.,  3.,  0.,  1.,  1.,  3.,
         2.,  2.,  3.,  0.,  2.,  2.,  1.,  1.,  2.,  1.,  0.,  3.,  1.,
         1.,  1.,  2.,  1.,  0.,  3.,  1.,  0.,  0.,  3.,  4.,  2.,  0.,
         1.,  1.,  3.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,
         0.,  0.,  0.,  1.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  2.,
         0.,  0.,  1.,  2.,  1.,  2.,  1.,  1.,  0.,  0.,  0.,  0.,  2.,
         2.,  1.,  2.,  1.,  0.,  1.,  2.,  1.,  0.,  0.,  2.,  2.,  3.,
         1.,  0.,  1.,  0.,  2.,  0.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,
         0.,  0.,  0.,  1.,  1.,  1.,  0.,  1.,  0.,  2.,  0.,  0.,  1.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  1.,  0.,  2.,  0.,  0.,  0.,  1.

## 新規感染者数

In [37]:
fig = plt.figure(figsize=(12,8))

plt.subplot(1,2,1) # (rows, columns, panel number)
plt.title("新規感染者数 Worst 50({})".format(latest_date))
plt.yticks(fontsize=9)
plt.xlabel("count (people)")
chart_data = country.sort_values("new_confirmed",ascending=False).head(50).sort_values("new_confirmed",ascending=True)
plt.barh(chart_data["Country/Region"],chart_data["new_confirmed"],label="positive")
plt.grid(axis="x")
mean = int(country.new_confirmed.mean())
plt.plot([mean,mean],[0,50], 'k--', lw=0.5)
plt.text(mean,50,"平均:{}".format(mean),ha="center",fontsize=9)
plt.legend()

plt.subplot(1,2,2) # (rows, columns, panel number)
plt.title("人口１０万人当たり新規感染者数 Worst 50({})".format(latest_date))
plt.yticks(fontsize=9)
plt.xlabel("count (people)")
chart_data = c_merged.sort_values("new positives per 100,000 population",ascending=False).head(50).sort_values("new positives per 100,000 population",ascending=True)
plt.barh(chart_data["Country/Region"],chart_data["new positives per 100,000 population"],label="positive")
plt.grid(axis="x")
mean = int(c_merged["new positives per 100,000 population"].mean())
plt.plot([mean,mean],[0,50], 'k--', lw=0.5)
plt.text(mean,50,"平均:{}".format(mean),ha="center",fontsize=9)
plt.legend()

fig.text(0,0
         ,"※感染者の情報提供:COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University,"\
         +"※人口の情報提供:WHO",
        fontsize=7,ha="left",va="bottom")
#fig.text(1,0.0,"※人口の情報提供:WHO",fontsize=8,ha="right")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [38]:
fig.savefig("out/covid-global-new-positives-latest-top.png")

### WORST50

In [39]:
country.sort_values("new_confirmed",ascending=False).head(50).reset_index()


,index,Country/Region,latest_confirmed2,latest_confirmed,new_confirmed
0,23,Brazil,18243483,18322760,79277
1,79,India,30134445,30183143,48698
2,37,Colombia,4060013,4092746,32733
3,6,Argentina,4350564,4374587,24023
4,144,Russia,5325940,5346005,20065
5,80,Indonesia,2053995,2072867,18872
6,162,South Africa,1877143,1895905,18762
7,185,United Kingdom,4700691,4716065,15374
8,181,US,33590481,33603148,12667
9,81,Iran,3140129,3150949,10820
